# Ambient Tracks

## Extraction

When I inserted the Mechwarrior 3 CD into a Windows PC, this message was shown:

![Select to choose what happens with enhanced audio CDs.](screenshot/enhanced-audio-cd.png)

And [enhanced audio CD](https://en.wikipedia.org/wiki/Enhanced_CD) usually means there is data and audio on the same disk. In fact there are two ambient/background tracks, roughly three minutes earch. Using [ExactAudioCopy](http://www.exactaudiocopy.de/) (EAC)\*, I ripped the tracks from US versions 1.0, 1.1, and 1.2, as well as German version 1.0 as Waveform Audio files (`*.wav`). This was mainly to make comparisons easier, but that didn't work out, as we will see. So I'd recommend compressing them using a lossy format like [AAC](https://en.wikipedia.org/wiki/Advanced_Audio_Coding) - a bitrate of 128 kilobytes should be plenty for ambient tracks. This produces file sizes around 3 MiB, so 10% of the original. (Lossless compression like [FLAC](https://xiph.org/flac/) produces ~40%.)

---

\* EAC is Windows only. Options on macOS are [RIP](https://sbooth.org/Rip/), [Max](https://sbooth.org/Max/), [XLD](https://tmkk.undo.jp/xld/index_e.html), or iTunes. There are *many* options on Linux, I suggest [Morituri](https://github.com/thomasvs/morituri).

## Next up

Skip over comparing the different MechWarrior 3 versions to [video files/cinematics](03-video-files.ipynb)

## Version comparison

Sadly, I could already see during the rip that comparison was going to be a bit trickier, since all the tracks had different [CRC](https://en.wikipedia.org/wiki/Cyclic_redundancy_check) codes. Another oddity is the fact the tracks aren't in the same order on different versions, or even the same lengths:

| Version | Track 2 | Track 3 |
|---------|---------|---------|
| v1.0 US | 3:11.69 | 3:06.06 |
| v1.0 DE | 3:06.06 | 3:13.69 |
| v1.1 US | 3:06.06 | 3:11.69 |
| v1.2 US | 3:06.06 | 3:11.69 |

They all sound indistinguishable for me, and the waveforms look the same, so it's probably fine. I'm unsure why this is though. It's most likely the result of the manufacturing process, but it could be deliberate DRM (specifically the German version uses Macrovision's SafeDisc DRM).

![A waveform plot of the shorter tracks, showing virtually indistinguishable waveforms. The data was resampled, but the amplitude is not re-normalised, so appears slightly quieter](analysis/audio-tracks-shorter.png)

![A waveform plot of the longer tracks, showing virtually indistinguishable waveforms. The data was resampled, but the amplitude is not re-normalised, so appears slightly quieter](analysis/audio-tracks-longer.png)

## How the waveform plots were made

While [LibROSA](http://librosa.github.io/librosa/) is a great library, I've decided to rely only on [SciPy](https://www.scipy.org/), since it [can also read WAV files](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html).

In [2]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


def time_ticker_format(x, pos=None):
    mins, secs = divmod(x, 60)
    return "{:.0f}:{:02.0f}".format(abs(mins), secs)


def plot_waveforms(tracks, save_name=None, resample_factor=40):
    """This function makes assumptions about the input data: stereo 44100 Hz 16-bit signed PCM"""
    data = []
    rates = []
    for track in tracks:
        rate, stereo = wavfile.read(track, mmap=True)
        samples, channels = stereo.shape
        assert channels == 2, "expecting stereo"
        mono = stereo.mean(1)
        # this is to make the data more resonable to plot
        resampled = resample(mono, int(np.ceil(mono.size / resample_factor)))
        data.append(resampled)
        rates.append(rate)

    rate = rates[0]
    assert all(rate == r for r in rates)

    count = len(tracks)
    fig, axis = plt.subplots(count, 1, figsize=(16, 4 * count))

    for ax, mono, name in zip(axis, data, tracks):
        samples = mono.size
        length = samples / rate
        time = np.linspace(0, length, num=samples)

        ax.plot(time, mono)
        ax.set_xlim(0, length)
        ax.xaxis.set_major_formatter(ticker.FuncFormatter(time_ticker_format))
        ax.xaxis.set_major_locator(ticker.MultipleLocator(20))
        ax.set_ylim(-(1 << 15), (1 << 15))  # signed 16-bit
        ax.yaxis.set_major_locator(ticker.NullLocator())
        ax.xaxis.set_label_text(name)

    fig.tight_layout()
    if save_name:
        plt.savefig(save_name)
        plt.close(fig)

In [3]:
# WARNING: can take a while to process (a minute or two)
plot_waveforms(
    [
        "audio/v1.0-us/03 Track03.wav",
        "audio/v1.0-de/02 Track02.wav",
        "audio/v1.1-us/02 Track02.wav",
        "audio/v1.2-us/02 Track02.wav",
    ],
    "analysis/audio-tracks-shorter.png",
)

In [4]:
# WARNING: can take a while to process (a minute or two)
plot_waveforms(
    [
        "audio/v1.0-us/02 Track02.wav",
        "audio/v1.0-de/03 Track03.wav",
        "audio/v1.1-us/03 Track03.wav",
        "audio/v1.2-us/03 Track03.wav",
    ],
    "analysis/audio-tracks-longer.png",
)

## Next up

[Video files/cinematics](03-video-files.ipynb)